In [61]:
import random
import math
import statistics

In [62]:
_memomask = {}
def random_hf(n):
    mask = _memomask.get(n)
    if mask is None:
        random.seed(n)
        mask = _memomask[n] = random.getrandbits(32)
    def myhash(x):
        return hash(x) ^ mask
    return myhash

def lsb(a):
    return a%2

In [78]:
class RoughEstimator:
    def __init__(self, n):
        self.n = n
        self.K_RE = math.ceil(max(8, math.log(n,2)/math.log(math.log(n,2),2)))
        self.counters_RE = [[-1 for i in range(self.K_RE)] for j in range(3)]
        hfs_index = random.sample([i for i in range(n)], k=9)
        self.hfs = [[random_hf(hfs_index[0]), random_hf(hfs_index[1]), random_hf(hfs_index[2])], 
                    [random_hf(hfs_index[3]), random_hf(hfs_index[4]), random_hf(hfs_index[5])],
                    [random_hf(hfs_index[6]), random_hf(hfs_index[7]), random_hf(hfs_index[8])]]
    
    def update(self, i):
        for j in range(3):
            self.counters_RE[j][self.hfs[j][2](self.hfs[j][1](i)%(self.K_RE**3))%self.K_RE] = max(self.counters_RE[j][self.hfs[j][2](self.hfs[j][1](i)%(self.K_RE**3))%self.K_RE], lsb(self.hfs[j][0](i)%(self.n-1)))
    
    def estimate(self):
        rou = 0.99 * (1 - math.exp(-1/3))
        Fs = [-1, -1, -1]
        for j in range(3):
            r = 0
            T = len([c for c in self.counters_RE[j] if c >= r])
            if T >= rou * self.K_RE:
                while (T >= rou * self.K_RE):
                    r += 1
                    T = len([c for c in self.counters_RE[j] if c >= r])
                Fs[j] = 2**r * self.K_RE
            else:
                Fs[j] = -1
        return statistics.median(Fs)

In [93]:
DATA = [random.randint(100,200) for i in range(1000)] #for sanity check

In [96]:
data = DATA

epsilon = 0.2
K = math.ceil(1/(epsilon ** 2))
counters = [(-1) for i in range(K)]
A = b = est = 0

failed = False
n = len(data)
hfs_index = random.sample([i for i in range(n)], k = 3)
hfs = [random_hf(hfs_index[0]), random_hf(hfs_index[1]), random_hf(hfs_index[2])]
RE = RoughEstimator(n)
for i in data:
    x = max(counters[hfs[2](hfs[1](i)%(K**3))%K], lsb(hfs[0](i)%(n-1))-b)
    A = A - math.ceil(math.log(2 + counters[hfs[2](hfs[1](i)%(K**3))%K],2)) + math.ceil(math.log(2+x,2))
    if A > 3 * K:
        failed = True
        print('FAIL')
        break
    counters[hfs[2](hfs[1](i)%(K**3))%K] = x
    RE.update(i)
    R = RE.estimate()
    if R > 2 ** est:
        est = math.log(R,2)
        b_new = max(0, est - math.log(K/32,2))
        for j in range(K):
            counters[j] = max(-1, counters[j] + b - b_new)
        b = b_new
        A = sum([math.log(c+2,2) for c in counters])
        
T = len([c for c in counters if c >= 0])
F0 = (2**b) * math.log(1-T/K) / math.log(1-1/K)

In [97]:
print('estimation result :', F0)

estimation result : -0.0
